In [3]:
import torch
import transformers
import trl
import json 
import torch
from datasets import Dataset, load_dataset
from trl import (setup_chat_format, 
                 DataCollatorForCompletionOnlyLM, 
                 SFTTrainer)
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig 
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          TrainingArguments, 
                          BitsAndBytesConfig, 
                          pipeline, 
                          StoppingCriteria)

In [4]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

In [5]:
model_id = "google/gemma-2-9b-it" 

# 모델과 토크나이저 불러오기 
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir= '/workspace/gemma-2-9b-it',
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager',
    #quantization_config=quant_config,
    #load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
from peft import LoraConfig, PeftModel
#merged_model = PeftModel.from_pretrained(base_model, "/workspace/multi_gpu_train/general_collator/model_output/checkpoint-21")
#merged_model = PeftModel.from_pretrained(base_model, "/workspace/multi_gpu_train/model_output/checkpoint-13")
#merged_model = PeftModel.from_pretrained(base_model, "/workspace/multi_gpu_train/8bit_quanta/model_output/checkpoint-1")
#merged_model = PeftModel.from_pretrained(base_model, "/workspace/multi_gpu_train/8bit_quanta/adapter_accelerate")
merged_model = PeftModel.from_pretrained(base_model, "/workspace/multi_gpu_train/qlora_fsdp/results/checkpoint-33")
finetuned_model = merged_model.merge_and_unload()

In [7]:
finetuned_model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((3584,), 

In [8]:
from transformers import StoppingCriteria, StoppingCriteriaList
user_token_id = tokenizer.encode("user", add_special_tokens=False)[0]

class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        super().__init__()
        self.stop_token_ids = stop_token_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in self.stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stop_words_ids = [user_token_id]
stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_token_ids=stop_words_ids)])

In [9]:
dataset = load_dataset("json", data_files="/workspace/train_dataset.jsonl")
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
test_dataset = dataset["test"].select(range(20))

In [10]:
tokenizer.decode(tokenizer.encode("<start_of_turn>"))

'<bos><start_of_turn>'

In [11]:
terminators = tokenizer.encode(tokenizer.additional_special_tokens[0])[1:]

In [12]:
terminators

[106]

In [13]:
tokenizer.additional_special_tokens[0]

'<start_of_turn>'

In [14]:
tokenizer.eos_token_id


1

### with generate func

In [16]:
for i in test_dataset:
    print("#########입력")
    messages = [i['messages'][0]]
    input_ids = tokenizer.apply_chat_template(messages,
                                              add_generation_prompt=True,
                                              return_tensors="pt").to(finetuned_model.device)
    print(tokenizer.decode(input_ids[0]))
    
    terminators = tokenizer.encode(tokenizer.additional_special_tokens[1])[1:]    
    output = finetuned_model.generate(
        input_ids, 
        max_new_tokens=512,
        #eos_token_id=terminators,
        do_sample=True,
        temperature=0.7,
        stopping_criteria=stopping_criteria,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # 생성된 텍스트를 디코딩합니다
    print("#####################3출력")
    generated_text = tokenizer.decode(output[0][len(input_ids[0]):], 
                                      #skip_special_tokens=True
                                     )
    print(generated_text)

#########입력
<bos><start_of_turn>user
음... 제가 동아리를 하면서 친구들이 다른 동아리와는 친하지 말라는 이야기를 들었어요.<end_of_turn>
<start_of_turn>model

#####################3출력
그런 상황이 있으셨군요. 그동안 어떻게 했나요?<end_of_turn>
<start_of_turn>user
#########입력
<bos><start_of_turn>user
네, 그래도 좀 어렵네요. 그래도 이야기를 해보려고 왔습니다.<end_of_turn>
<start_of_turn>model

#####################3출력
네, 좋습니다. 얘기해보세요.<end_of_turn>
<start_of_turn>user
#########입력
<bos><start_of_turn>user
저는 회사에서 팀원들과 잘 지내지 못하고 있습니다.<end_of_turn>
<start_of_turn>model

#####################3출력
그러시군요. 그래서 어떤 고민을 가지고 계시나요?<end_of_turn>
<start_of_turn>user
#########입력
<bos><start_of_turn>user
안녕하세요. 제가 친구들과 대화할 때 어색해져요. 처음 만나는 사람들과 대화할 때도 어색해요. 이런 부분이 부담스러워서 상담을 요청했어요.<end_of_turn>
<start_of_turn>model

#####################3출력
네, 그렇군요. 그렇다면, 대화할 때 어색해지는 부분을 좀 더 자세히 말씀해주세요.<end_of_turn>
<start_of_turn>user
#########입력
<bos><start_of_turn>user
요즘에 학교에서 교복을 잘 안입고 다니는 것 같아요. 원래 학교에서 교복을 입어야 하는 규정은 있는데 제가 속상하고 괴로운데 말이죠. 저만 교복을 안입으면 다른 학생들에게 눈치보일 것 같아서 걱정이 되지만 불쾌하고 짜증스러운 느낌이 더 

## with pipeline

In [7]:
pipe = pipeline(
    "text-generation",
    model=finetuned_model,
    tokenizer=tokenizer,
    device_map="auto",
    return_full_text=False,
    do_sample=True,
    max_new_tokens=512,
    temperature=0.7,
    
)

In [8]:
responses = []
for step,i in enumerate(test_dataset[:12]['messages']):
    
    input_text = i[0]['content']
    print("@@ sample",step)
    print(input_text)
    print("###################답변################")
    output = pipe(
        input_text ,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        #stopping_criteria=stopping_criteria,
        pad_token_id=tokenizer.eos_token_id
    )
    
    print(output[0]["generated_text"])
    responses.append(output[0]["generated_text"])
    
import os 
import pickle
os.makedirs('result', exist_ok=True)
with open('result'+"/responses_after_train_multi.pkl","wb") as f:
    pickle.dump(responses, f)

@@ sample 0
음... 제가 동아리를 하면서 친구들이 다른 동아리와는 친하지 말라는 이야기를 들었어요.
###################답변################


KeyboardInterrupt: 

## 결과비교

In [10]:
import pickle

with open('result'+"/responses_before_train.pkl",'rb') as f:
    list_before = pickle.load(f)
with open('result'+"/responses_after_train_multi.pkl",'rb') as f:
    list_after = pickle.load(f)
    
for step,i in enumerate(test_dataset[:12]['messages']):
    
    input_text = i[0]['content']
    print("@@ sample",step)
    print(input_text)
    print("#before###################답변################")
    print(list_before[step])
    print(" ")
    print("#after###################답변################")
    print(list_after[step])
    

@@ sample 0
음... 제가 동아리를 하면서 친구들이 다른 동아리와는 친하지 말라는 이야기를 들었어요.
#before###################답변################


IndexError: list index out of range

In [11]:
list_before

[]

In [14]:
for i in list_after:
    print(i)
    print("######################3 ")

 그리고 내 동아리에서 가장 친한 친구들이 1-2명 정도이고 나머지는 그냥 나랑은 상관없어 보여요. 그러다가 지금까지 그냥 괜찮았는데 어느덧 학교가 끝나고 집에 가면 정말 힘들어져요. 이제는 학교에 가고 싶지 않고 동아리에도 가고 싶지 않아요. 하루종일 학교에 있으면 힘들어지고 동아리에 가면 너무 지쳐서 집에 가고 싶어져요.
model
그러셨군요. 그렇다면, 어떤 일이 있었을까요?
user
######################3 

model
그러시군요. 어떤 일을 해결하려고 하시는 건가요?
user
######################3 
 제가 가진 성격 때문에 팀원들과 관계를 맺기가 어렵고, 이 때문에 정신적으로 힘들어지고 있습니다. 이러한 문제를 해결하기 위해서 제가 할 수 있는 것이 뭐가 있을까요?
model
회사에서 팀원들과 잘 지내지 못하고 계시다고 하셨는데, 그런 상황에서 힘드시겠습니다. 저는 이러한 상황에서 어떤 해결책이 있을지에 대해서 제가 알고 있는 것을 말씀드리겠습니다. 이후에 제가 추천하는 방법이 여러분에게 어울리는지는 제가 알 수 없습니다. 그러나 여러분이 이미 진행했던 해결책들 중에서 제가 추천하는 방법이 어울리는지 생각해보시면 좋습니다.
user
######################3 

model
네, 그렇죠? 그런 상황이 어려울 수 있습니다.
user
######################3 

model
학교에서 교복을 입어야 하는 것은 규칙이니까, 학교나 선생님들께서는 학교규칙에 따라 규칙을 잘 지켜달라고 이야기하겠죠. 교복은 학교에서 공통적으로 입는 옷이니까, 다른 학생들과 차별없이 다닐 수 있는 옷이죠. 내담자님이 학교에서 교복을 잘 안입고 다니는 것 같아요. 이게 어떤 이유 때문일까요?
user
######################3 
 제가 지금은 학교에 가고 있고, 학교생활에서도 괜찮은 편인데 저는 그래도 자꾸 스트레스가 쌓이는 것 같아요.
model
저는 이해합니다. 스트레스가 많아지면 집중력도 떨